In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import cv2
import re
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import librosa

from PIL import Image
import IPython.display as ipd

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from tqdm import tqdm
from utils import io, train
from utils import feature_engineering as fe

In [3]:
videos_folder = "../data/videos"
gt_folder = "../data/gt_annotations"
temp_folder = "../data/tmp/"

In [4]:
episode_names, video_paths, gt_df = io.get_init_vars(gt_folder, videos_folder)

In [77]:
import utils.feature_engineering as fe

# Haralick texture

In [127]:
texture_feat = fe.extract_haralick_texture(video_paths, episode_names)
pickle.dump(texture_feat, open(os.path.join('../data/features/', 'texture_feats.pkl'), 'wb'))


  0%|          | 0/38707 [00:00<?, ?it/s]

100%|██████████| 38499/38499 [1:06:07<00:00,  9.70it/s]


In [155]:
textures = pickle.load(open(os.path.join('../data/features/', 'texture_feats.pkl'), 'rb'))

# Pig Color

In [185]:
import utils.feature_engineering as fe

pig_color_feats = {}
for episode in episode_names:
    feats = fe.get_pig_pixels(video_paths[episode])
    pig_color_feats[episode] = feats
    
pickle.dump(pig_color_feats, open(os.path.join('../data/features/', 'pig_color_feats.pkl'), 'wb'))

../data/videos/Muppets-02-01-01.avi
Loaded video with resolution 720x544
FPS = 25.0, Frame Count = 38682, Codec = FMP4


100%|██████████| 38682/38682 [04:24<00:00, 146.19it/s]


../data/videos/Muppets-02-04-04.avi
Loaded video with resolution 720x544
FPS = 25.0, Frame Count = 38707, Codec = FMP4


100%|██████████| 38707/38707 [04:12<00:00, 153.58it/s]


../data/videos/Muppets-03-04-03.avi
Loaded video with resolution 704x544
FPS = 25.0, Frame Count = 38499, Codec = FMP4


100%|██████████| 38499/38499 [03:53<00:00, 164.83it/s]


In [186]:
pig_color_feats = pickle.load(open(os.path.join('../data/features/', 'pig_color_feats.pkl'), 'rb'))

In [188]:
texture_feats = pickle.load(open(os.path.join('../data/features/', 'texture_feats.pkl'), 'rb'))
pig_color_feats = pickle.load(open(os.path.join('../data/features/', 'pig_color_feats.pkl'), 'rb'))

In [251]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.naive_bayes import GaussianNB

# train on episode 2, test on episode 0
train_episode = episode_names[2]
test_episode = episode_names[0]

# get features
texture_train = texture_feats[train_episode]['texture']
texture_test = texture_feats[test_episode]['texture']
list_of_train_color_feats = []
list_of_test_color_feats = []
for feat in (pig_color_feats[train_episode].keys()):
    list_of_train_color_feats.append(pig_color_feats[train_episode][feat])
    list_of_test_color_feats.append(pig_color_feats[test_episode][feat])

pig_color_train = np.array(list_of_train_color_feats).T
pig_color_test = np.array(list_of_test_color_feats).T

# merge features
train_feats = np.hstack((texture_train, pig_color_train))
test_feats = np.hstack((texture_test, pig_color_test))

# get target
target = "Cook"
gt_train = gt_df[gt_df['episode'] == train_episode][target]
gt_test = gt_df[gt_df['episode'] == test_episode][target]

# train model
scaler = StandardScaler()
train_feats = scaler.fit_transform(train_feats)
test_feats = scaler.transform(test_feats)
dt = LinearSVC(class_weight='balanced', max_iter=10000, C=0.00001)
# dt = GaussianMixture(n_components=2, covariance_type='full')
# dt = GaussianNB()
dt.fit(train_feats, gt_train)


/home/lexi/miniconda3/envs/SimilarityModeling/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC(C=1e-05, class_weight='balanced', max_iter=10000)

In [252]:
# evaluate on test set
pred = dt.predict(test_feats)
gt = gt_test

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# evaluate metrics and print them in nice format
acc = accuracy_score(gt, pred)
precision, recall, f1, _ = precision_recall_fscore_support(gt, pred, average='binary')
print('Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f} | F1: {:.4f}'.format(acc, precision, recall, f1))

Accuracy: 0.6445 | Precision: 0.1107 | Recall: 0.9033 | F1: 0.1973
